## Importing needed modules and applying configs

In [1]:
import sys
import os
sys.path.insert(0, 'C:/Users/Lenovo/Desktop/acronis test/task/power_bi_reporting_solution/src')
import sql_profiler
import warnings
warnings.filterwarnings('ignore')

## Initiating sql_profiler custom module`s ssms_profiler class with needed params

In [2]:
analysis = sql_profiler.ssms_profiler('SQL_server',os.environ['MYSQLSERVER_PASS'])

## Applying data profiling methods on defined database`s all tables and saving results as excel file into working directory

In [3]:
# analysis.multiple_dataset_apply_profiling()

## Using apply_sql_query method from custom module to bring SQL query results

### Query-1 : Which teams had the shortest and the longest average duration of chats?

In [43]:
analysis.apply_sql_query('WITH CTE_duration (case_id, createddate, enddate, duration) \
                         AS \
                         ( \
                         SELECT case_id, \
                         createddate, enddate, \
                         DATEDIFF(second, createddate, enddate) AS duration \
                         FROM dbo.test_LiveChatTranscript \
                         ), \
                         CTE_chat_group_team_duration (owner_team, average_chat_duration) \
                         AS \
                         ( \
                         SELECT owner_team , AVG(duration) as average_chat_duration  \
                         FROM CTE_duration \
                         JOIN dbo.test_cases cases \
                         ON CTE_duration.case_id = cases.caseid \
                         GROUP BY owner_team \
                         ), \
                         CTE_chat_ranks (owner_team, average_chat_duration, rank_chat_dur) \
                         AS \
                         ( \
                         SELECT owner_team, average_chat_duration, RANK() OVER (ORDER BY average_chat_duration) as rank_chat_dur \
                         FROM CTE_chat_group_team_duration \
                         ) \
                         SELECT owner_team FROM CTE_chat_ranks WHERE rank_chat_dur = 1 OR rank_chat_dur IN (SELECT  MAX(rank_chat_dur) FROM CTE_chat_ranks)'
                         )

,owner_team
0,O
1,A


### Query-2 : Which team had the lowest average response time between first incoming email and the next call. What was the average for this team?

In [128]:
analysis.apply_sql_query('WITH CTE_incoming_mails (caseid, sent, incoming,rank_email_time_by_case) \
                         AS \
                         ( \
                         SELECT caseid, \
                         sent, incoming , RANK() OVER (PARTITION BY caseid ORDER BY sent ASC) as rank_email_time_by_case \
                         FROM dbo.test_Emails WHERE incoming = 1 \
                         ), \
                         CTE_first_mails_filtered (caseid, sent) \
                         AS \
                         ( \
                         SELECT caseid, sent FROM CTE_incoming_mails \
                         WHERE rank_email_time_by_case = 1 \
                         ), \
                         CTE_mail_call_cases_joined (caseid, email_sent_time, call_start_time ,owner_team) \
                         AS \
                         ( \
                         SELECT e.caseid, e.sent AS email_sent_time, c.started_time AS call_start_time, \
                         ca.owner_team AS owner_team FROM CTE_first_mails_filtered e \
                         JOIN dbo.test_cases ca ON ca.caseid = e.caseid \
                         JOIN dbo.test_calls c ON c.caseid = e.caseid \
                         ), \
                         CTE_ranked_calls (caseid, email_sent_time, call_start_time , rank_of_call, owner_team) \
                         AS \
                         ( \
                         SELECT caseid, email_sent_time, call_start_time, RANK() OVER (PARTITION BY email_sent_time ORDER BY call_start_time ASC) as rank_call_start, \
                         owner_team \
                         FROM CTE_mail_call_cases_joined \
                         ), \
                         CTE_first_calls_filtered (email_sent_time, call_start_time , rank_of_call, owner_team) \
                         AS \
                         ( \
                         SELECT email_sent_time, call_start_time , rank_of_call, owner_team FROM CTE_ranked_calls \
                         WHERE rank_of_call = 1 \
                         ), \
                         CTE_aggregated_response (owner_team, response_time) \
                         AS \
                         ( \
                         SELECT owner_team, \
                         AVG(DATEDIFF(second, email_sent_time, call_start_time)) AS response_time  \
                         FROM  CTE_first_calls_filtered \
                         GROUP BY owner_team \
                         ) \
                         SELECT TOP(1) owner_team, response_time FROM CTE_aggregated_response \
                         ORDER BY response_time ASC ')

,owner_team,response_time
0,J,511299
